In [1]:
import boto3

In [2]:
%%writefile Dockerfile

FROM python:3.9-slim-buster

COPY requirements.txt /opt/ml/requirements.txt
COPY code/preprocess.py /opt/ml/code/preprocess.py
COPY code/train.py /opt/ml/code/train.py

RUN pip3 install -r /opt/ml/requirements.txt

ENV SAGEMAKER_PROGRAM preprocess.py
ENV SAGEMAKER_PROGRAM train.py
ENV PYTHONUNBUFFERED=TRUE

ENTRYPOINT ["python3", "/opt/ml/code/train.py"]

Overwriting Dockerfile


In [3]:
account_id = boto3.client('sts').get_caller_identity().get('Account')
region = boto3.Session().region_name
ecr_repository = 'demo_prophet'
tag = ':latest'
processing_repository_uri = '{}.dkr.ecr.{}.amazonaws.com/{}'.format(account_id, region, ecr_repository + tag)

In [4]:
# Create ECR repository and push docker image
!docker build -t $ecr_repository .

Sending build context to Docker daemon  152.1kB
Step 1/9 : FROM python:3.9-slim-buster
3.9-slim-buster: Pulling from library/python

b88d5577: Pulling fs layer 
16e23423: Pulling fs layer 
da260408: Pulling fs layer 
c79126f6: Pulling fs layer 
Digest: sha256:320a7a4250aba4249f458872adecf92eea88dc6abd2d76dc5c0f01cac9b53990[5A
Status: Downloaded newer image for python:3.9-slim-buster
 ---> c84dbfe3b8de
Step 2/9 : COPY requirements.txt /opt/ml/requirements.txt
 ---> 7edd8862d744
Step 3/9 : COPY code/preprocess.py /opt/ml/code/preprocess.py
 ---> 5b9b79d96d42
Step 4/9 : COPY code/train.py /opt/ml/code/train.py
 ---> e89a5ba6f79a
Step 5/9 : RUN pip3 install -r /opt/ml/requirements.txt
 ---> Running in f1aeec01b6a0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 51.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 50.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
!aws ecr get-login-password --region {region} | docker login --username AWS --password-stdin {account_id}.dkr.ecr.{region}.amazonaws.com

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [6]:
!aws ecr create-repository --repository-name $ecr_repository

{
    "repository": {
        "repositoryArn": "arn:aws:ecr:us-east-1:654654222480:repository/demo_prophet",
        "registryId": "654654222480",
        "repositoryName": "demo_prophet",
        "repositoryUri": "654654222480.dkr.ecr.us-east-1.amazonaws.com/demo_prophet",
        "createdAt": 1723878392.229,
        "imageTagMutability": "MUTABLE",
        "imageScanningConfiguration": {
            "scanOnPush": false
        },
        "encryptionConfiguration": {
            "encryptionType": "AES256"
        }
    }
}


In [7]:
# aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1
# if [ $? -ne 0 ]
# then
# aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
# fi

In [8]:
!docker tag {ecr_repository + tag} $processing_repository_uri

In [9]:
!docker push $processing_repository_uri

The push refers to repository [654654222480.dkr.ecr.us-east-1.amazonaws.com/demo_prophet]

3fac1990: Preparing 
dc1a608e: Preparing 
82320d95: Preparing 
dcbaed91: Preparing 
a27560c1: Preparing 
037e08b3: Preparing 
eede8d6e: Preparing 
55769c5e: Preparing 
3fac1990: Pushed   657.9MB/627.5MB2KPushing  309.3MB/627.5MBPushing  620.8MB/627.5MBlatest: digest: sha256:f5ac555ba55cfa0711c1d45e789d2b8bab9f2583d5b2ddc2873d56838b46ee3e size: 2204
